In [ ]:
!git clone https://github.com/IZ-f-AI-ss3/Denoising_in_score_based_generative_models.git
%cd /content/Denoising_in_score_based_generative_models/

In [ ]:
!git pull

In [ ]:
!pip install gdown
!pip install tensorboardX  # pyyaml

import sys
sys.path.append('/content/Denoising_in_score_based_generative_models')

!gdown https://drive.google.com/uc?id=1BF2mwFv5IRCGaQbEWTbLlAOWEkNzMe5O
!unzip cifar10.zip -d /content/Denoising_in_score_based_generative_models/run/logs
!unzip run.zip -d /content/Denoising_in_score_based_generative_models/


In [ ]:
!python main.py --runner AnnealRunner --test -o samples  --doc celeba --sampling half_initial_denoising

# !python main.py --runner AnnealRunner --heavy_test -o samples --doc cifar10 --n_samples 200 --sampling_type half_denoising # ordinary #50000

# Warning : it took 15 minutes for 2 batchs (200 samples) and we need 50k samples ???

In [ ]:
import shutil

shutil.make_archive(
    "half_denoising",
    "zip",
    "/content/Denoising_in_score_based_generative_models/samples/half_denoising"
)


In [ ]:
from google.colab import files
files.download("half_denoising.zip")

In [ ]:
!zip -r samples.zip /content/Denoising_in_score_based_generative_models/samples/

In [ ]:
from google.colab import files
files.download('samples.zip')

In [ ]:
!pip install torch-fidelity

In [ ]:
# !mkdir -p /content/samples_ordinary_mnist_10k.zip
# !unzip /content/samples_ordinary_mnist_10k.zip -d /content/
!unzip /content/whole_celeba_kaggle.zip -d /content/

# !unzip samples_debug.zip

In [ ]:
from torchvision.utils import save_image
from torchvision import transforms, datasets
import os

tran_transform = transforms.Compose([
                transforms.Resize(28), # 32 for the others
                transforms.ToTensor()
            ])
mnist_dataset = datasets.MNIST(
    root='/content/data/mnist',
    train=True,
    download=True,
    transform=tran_transform
)

mnist_folder = '/content/data/mnist_for_fid'
os.makedirs(mnist_folder, exist_ok=True)

for i, (img, _) in enumerate(mnist_dataset):
    save_image(img, os.path.join(mnist_folder, f"{i}.png"))

In [ ]:
from torchvision.utils import save_image
from torchvision import transforms, datasets
import os

tran_transform = transforms.Compose([
                transforms.Resize(28), # 32 for the others
                transforms.ToTensor()
            ])
cifar_dataset = datasets.CIFAR10(
    root='/content/data/cifar',
    train=True,
    download=True,
    transform=tran_transform
)

cifar_folder = '/content/data/cifar_for_fid'
os.makedirs(mnist_folder, exist_ok=True)

for i, (img, _) in enumerate(mnist_dataset):
    save_image(img, os.path.join(mnist_folder, f"{i}.png"))


In [ ]:
import os
import zipfile
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tqdm import tqdm

# 1. Unzip the file
zip_path = '/content/whole_celeba_kaggle.zip'
extract_path = '/content/celeba_extracted_3'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

transform = transforms.Compose([
    transforms.CenterCrop(140),
    transforms.Resize(32),
    transforms.ToTensor(),
])

dataset = datasets.ImageFolder(
    root=extract_path,
    transform=transform
)

celeba_folder = '/content/data/celeba_for_fid_3'
os.makedirs(celeba_folder, exist_ok=True)

# 5. Save images
print(f"Saving images to {celeba_folder}...")
for i in tqdm(range(len(dataset)), desc="Processing"):
    img, _ = dataset[i]
    save_image(img, os.path.join(celeba_folder, f"{i}.png"))

print(f"Success! {len(dataset)} images saved.")

In [ ]:
# import os
# import torch
# from torchvision import datasets, transforms
# from torchvision.utils import save_image
# from tqdm import tqdm # Useful for tracking progress

# # 1. Define Transforms
# transform = transforms.Compose([
#     transforms.CenterCrop(140),
#     transforms.Resize(32),
#     transforms.ToTensor(),
# ])

# # 2. Load CelebA dataset
# # Note: If download=True fails, you must manually download the files
# # (img_align_celeba.zip and identity_CelebA.txt, etc.) into /content/data/celeba/
# try:
#     celeba_dataset = datasets.CelebA(
#         root='/content/data',
#         split='train',
#         transform=transform,
#         download=True
#     )
# except RuntimeError as e:
#     print(f"Download failed: {e}")
#     print("Tip: CelebA download limits are often reached. You may need to manual download.")

# # 3. Setup output folder
# celeba_folder = '/content/data/celeba_for_fid'
# os.makedirs(celeba_folder, exist_ok=True)

# # 4. Save images (using tqdm to see progress)
# print(f"Saving images to {celeba_folder}...")
# for i, (img, _) in enumerate(tqdm(celeba_dataset)):
#     save_image(img, os.path.join(celeba_folder, f"{i}.png"))


In [ ]:
import torch
import torch_fidelity
import os
from torchvision import datasets, transforms

file_path_ordinary = "/content/kaggle/working/Denoising_in_score_based_generative_models/samples/ordinary/"
# file_path_half_denoising = "/content/celeba-half-denoising-7501"

print(f"Metrics calculation using image folder: {file_path_ordinary}")

print("\n--- Results for Ordinary Sampling ---")
metrics_ord = torch_fidelity.calculate_metrics(
    input1=file_path_ordinary,
    input2=celeba_folder,
    cuda=True,
    fid=True,
    isc=True,
    verbose=True
)

print("-------------------------------------------------------")
print('-------------------------------------------------------')
print(f"FID: {metrics_ord['frechet_inception_distance']:.4f}")
print(f"IS:  {metrics_ord['inception_score_mean']:.4f} ± {metrics_ord['inception_score_std']:.4f}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import torch
# import torch_fidelity
# from torch.utils.data import TensorDataset

# _original_load = torch.serialization.load
# def fixed_load(*args, **kwargs):
#     kwargs.setdefault("weights_only", False)
#     return _original_load(*args, **kwargs)

# torch.load = fixed_load

# file_path_ordinary = "/content/kaggle/working/Denoising_in_score_based_generative_models/samples/ordinary"
# # file_path_ordinary = "/content/Denoising_in_score_based_generative_models/samples/ordinary.pt"
# # file_path_half = "/content/Denoising_in_score_based_generative_models/samples/half_denoising.pt"

# print("Loading sample tensors from .pt files...")

# samples_ord_tensor = torch.load(file_path_ordinary)
# # samples_half_tensor = torch.load(file_path_half)

# # if samples_ord_tensor.dtype != torch.uint8:
# #     samples_ord_tensor = (samples_ord_tensor * 255).clamp(0, 255).to(torch.uint8)
# # if samples_half_tensor.dtype != torch.uint8:
# #     samples_half_tensor = (samples_half_tensor * 255).clamp(0, 255).to(torch.uint8)

# if samples_ord_tensor.dim() == 4 and samples_ord_tensor.size(1) == 1:
#     samples_ord_tensor = samples_ord_tensor.repeat(1, 3, 1, 1)
# elif samples_ord_tensor.dim() != 4 or samples_ord_tensor.size(1) not in (1, 3):
#     raise ValueError(f"Unexpected tensor shape for image data: {samples_ord_tensor.shape}")


# ds_ordinary = TensorDataset(samples_ord_tensor)
# # ds_half = TensorDataset(samples_half_tensor)



# print("\n--- Results for Ordinary Sampling ---")
# metrics_ord = torch_fidelity.calculate_metrics(
#     input1= ds_ordinary,
#     input2="cifar10-train",
#     cuda=True,
#     fid=True,
#     isc=True,
#     verbose=False
# )
# print(f"FID: {metrics_ord['frechet_inception_distance']:.4f}")
# print(f"IS:  {metrics_ord['inception_score_mean']:.4f} ± {metrics_ord['inception_score_std']:.4f}")

# # print("\n--- Results for Half-Denoising ---")
# # metrics_half = torch_fidelity.calculate_metrics(
# #     input1=ds_half,
# #     input2="cifar10-train",
# #     cuda=True,
# #     fid=True,
# #     isc=True,
# #     verbose=False
# # )
# # print(f"FID: {metrics_half['frechet_inception_distance']:.4f}")
# # print(f"IS:  {metrics_half['inception_score_mean']:.4f} ± {metrics_half['inception_score_std']:.4f}")

In [ ]:
# import torch
# import torch_fidelity

# _original_load = torch.load

# def fixed_load(*args, **kwargs):
#     # Inject weights_only=False if it's missing
#     if 'weights_only' not in kwargs:
#         kwargs['weights_only'] = False
#     return _original_load(*args, **kwargs)

# torch.load = fixed_load

# path_ordinary = "/content/content/Denoising_in_score_based_generative_models/samples/ordinary"
# path_half = "/content/content/Denoising_in_score_based_generative_models/samples/half_denoising"

# print("--- Results for Ordinary Sampling ---")
# metrics_ord = torch_fidelity.calculate_metrics(
#     input1=path_ordinary,
#     input2="cifar10-train",
#     cuda=True,
#     fid=True,
#     isc=True,
#     verbose=False
# )
# print(f"FID: {metrics_ord['frechet_inception_distance']:.4f}")
# print(f"IS:  {metrics_ord['inception_score_mean']:.4f} ± {metrics_ord['inception_score_std']:.4f}")

# print("\n--- Results for Half-Denoising ---")
# metrics_half = torch_fidelity.calculate_metrics(
#     input1=path_half,
#     input2="cifar10-train",
#     cuda=True,
#     fid=True,
#     isc=True,
#     verbose=False
# )
# print(f"FID: {metrics_half['frechet_inception_distance']:.4f}")
# print(f"IS:  {metrics_half['inception_score_mean']:.4f} ± {metrics_half['inception_score_std']:.4f}")

# # # 1. Calculate for Ordinary Sampling
# # print("--- Results for Ordinary Sampling ---")
# # !fidelity --gpu 0 --fid --isc --input1 "/content/content/Denoising_in_score_based_generative_models/samples/ordinary" --input2 cifar10-train

# # # 2. Calculate for Half-Denoising Sampling
# # print("\n--- Results for Half-Denoising ---")
# # !fidelity --gpu 0 --fid --isc --input1 "/content/content/Denoising_in_score_based_generative_models/samples/half_denoising" --input2 cifar10-train